Integrate models generated by Carveme with required reactions and metabolites.

In [ ]:
import os
import shutil
import cobra as cb

config = cb.Configuration()
config.solver = 'cplex'

In [ ]:
model_dir_in = '../data/models/gapfilled/'
model_dir_out = '../data/models/gapfilled_and_amended/'

In [ ]:
for m in os.listdir(model_dir_in):
    print(m + ': ')
    if m.strip('.xml') == 'bin_9_metaspades':
        # add acetate transport to cytoplasm through periplasm
        model = cb.io.read_sbml_model(model_dir_in + m)
        ac_e = cb.Metabolite('ac_e', formula=model.metabolites.ac_c.formula, name='Acetate', compartment='C_e')
        ac_p = cb.Metabolite('ac_p', formula=model.metabolites.ac_c.formula, name='Acetate', compartment='C_p')
        e_reaction = cb.Reaction('EX_ac_e')
        e_reaction.name = 'Acetate exchange'
        e_reaction.subsystem = ''
        e_reaction.lower_bound = -1000.  # This is the default
        e_reaction.upper_bound = 1000.
        e_reaction.add_metabolites({ac_e: -1.0})
        e2p_reaction = cb.Reaction('ACtex')
        e2p_reaction.name = 'Acetate transport via diffusion (extracellular to periplasm)'
        e2p_reaction.subsystem = ''
        e2p_reaction.lower_bound = -1000.  # This is the default
        e2p_reaction.upper_bound = 1000.
        e2p_reaction.add_metabolites({ac_e: -1.0, ac_p: 1.0})
        p2c_reaction = cb.Reaction('ACt2pp')
        p2c_reaction.name = 'Acetate transport in via proton symport (periplasm)'
        p2c_reaction.subsystem = ''
        p2c_reaction.lower_bound = -1000.  # This is the default
        p2c_reaction.upper_bound = 1000.
        p2c_reaction.add_metabolites({model.metabolites.h_p: -1.0, ac_p: -1.0, model.metabolites.h_c: 1.0, model.metabolites.ac_c: 1.0})
        # add glycine reductive pathway reactions
        GLYR = cb.Reaction('GLYR')
        GLYR.name = 'Glycine reductase'
        GLYR.subsystem = ''
        GLYR.lower_bound = -1000.
        GLYR.upper_bound = 1000.
        GLYR.add_metabolites({model.metabolites.actp_c: -1.0, model.metabolites.nh4_c: -1.0, model.metabolites.trdox_c: -1.0, \
            model.metabolites.gly_c: 1.0, model.metabolites.h_c: 1.0, model.metabolites.pi_c: 1.0, model.metabolites.trdrd_c: 1.0})
        SDA = cb.Reaction('SDA')
        SDA.name = 'Serine dehydratase'
        SDA.subsystem = ''
        SDA.lower_bound = -1000.
        SDA.upper_bound = 1000.
        SDA.add_metabolites({model.metabolites.ser__L_c: -1.0, model.metabolites.h_c: -1.0, \
            model.metabolites.pyr_c: 1.0, model.metabolites.nh4_c: 1.0})

        model.add_reactions([e_reaction, e2p_reaction, p2c_reaction, GLYR, SDA])
        cb.io.write_sbml_model(model, model_dir_out + m)
    
    else:
        shutil.copyfile(model_dir_in + m, model_dir_out + m)